In [4]:
import cv2
import numpy as np

# Function to detect hand gestures based on contours
def detect_gesture(frame, hsv_frame):
    # Define the range of skin color in HSV (you might need to adjust this)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)

    # Create a mask based on the skin color
    mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    
    # Apply some morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=3)
    mask = cv2.erode(mask, kernel, iterations=2)

    # Find contours of the hand in the mask
    contours, hierarchy = cv2.findContours(cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # If contours are found, process the largest one
    if contours:
        # Sort contours by area and get the largest one
        contour = max(contours, key=cv2.contourArea)

        # Get the convex hull of the contour to smooth out the shape
        hull = cv2.convexHull(contour)

        # Calculate the convexity defects of the hand
        defects = cv2.convexityDefects(contour, cv2.convexHull(contour, returnPoints=False))

        # Draw the contour and hull
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

        # Count the number of fingers (simple gesture recognition)
        finger_count = 0
        if defects is not None:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])

                # Draw the defects
                cv2.line(frame, start, end, [0, 0, 255], 2)
                cv2.circle(frame, far, 5, [0, 0, 255], -1)

                # Count defects (fingers) based on angle
                if d > 10000:
                    finger_count += 1

        # Display the number of fingers detected
        cv2.putText(frame, f'Fingers: {finger_count + 1}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    return frame

# Function to display the live camera feed and apply hand gesture detection
def main():
    # Initialize the webcam
    cap = cv2.VideoCapture(1)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        # Convert the frame to HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Detect hand gestures
        frame = detect_gesture(frame, hsv_frame)

        # Display the frame with gesture recognition
        cv2.imshow('Hand Gesture Recognition', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

# Call the main function to start the program
if __name__ == "__main__":
    main()


Failed to grab frame.


[ WARN:0] VIDEOIO(V4L2:/dev/video1): can't open camera by index
